# Imports 

In [2]:
import csv  # for slang
import os
import re  # regex
import string  # punct
from pprint import pprint
!pip install emoji --upgrade
import emoji  # for emoji
import gensim
import keras
import lightgbm as lgb
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from gensim.models import Word2Vec
from IPython.display import Image
from matplotlib import pyplot as plt
from nltk.corpus import stopwords  # stopwords
from nltk.stem import PorterStemmer  # stemming
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn import svm, tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier, RandomForestRegressor,
                              StackingClassifier)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import (accuracy_score, auc, average_precision_score,
                             brier_score_loss, classification_report,
                             confusion_matrix, f1_score, fbeta_score,
                             make_scorer, plot_precision_recall_curve,
                             precision_recall_curve, precision_score,
                             recall_score, roc_auc_score, roc_curve)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     cross_val_score, train_test_split)
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.svm import SVC  # "Support vector classifier"
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from google.colab import files
uploaded = files.upload()
import nltk
nltk.download('stopwords')

%matplotlib inline

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Saving slang.txt to slang.txt
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Homemade Classes and Functions

In [3]:
# Clean Text Class

class CleanText(BaseEstimator, TransformerMixin):
    
    def remove_mentions(self, input_text):
        '''
        Remove mentions, like @Mplamplampla
        '''
        return re.sub(r'@+', '', input_text)
    
    def remove_urls(self, input_text):
        '''
        Remove the urls mention in a tweet
        '''
        input_text  = ' '.join([w for w in input_text.split(' ') if '.com' not in w])
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        input_text = emoji.demojize(input_text)
        input_text = input_text.replace('_','')
        input_text = input_text.replace(':','')
        return input_text
    
    def possessive_pronouns(self, input_text):
        '''
        Remove the possesive pronouns, because otherwise after tokenization we will end up with a word and an s
        Example: government's --> ["government", "s"]
        '''
        return input_text.replace("'s", "")
    
    def characters(self, input_text):
        '''
        Remove special and redundant characters that may appear on a tweet and that don't really help in our analysis
        '''
        input_text = input_text.replace("\r", " ") # Carriage Return
        input_text = input_text.replace("\n", " ") # Newline
        input_text = " ".join(input_text.split()) # Double space
        input_text = input_text.replace('"', '') # Quotes
        return input_text
    
    def remove_punctuation(self, input_text):
        '''
        Remove punctuation and specifically these symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        '''
        punct = string.punctuation # string with all the punctuation symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        '''
        Remove numbers
        '''
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        '''
        Convert all the sentences(words) to lowercase
        '''
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        '''
        Remove stopwords (refers to the most common words in a language)
        '''
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        '''
        Reduce the words to their stem
        '''
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def encode_decode(self, input_text):
        '''
        Remove weird characters that are result of encoding problems
        '''
        return  " ".join([k.encode("ascii", "ignore").decode() for k in input_text.split(" ")])
    
    
    def translator(self, input_text):
        '''
        Transform abbrevations to normal words
        Example: asap --> as soon as possible
        '''
        input_text = input_text.split(" ")
        j = 0
        for _str in input_text:
            # File path which consists of Abbreviations.
            fileName = "/content/slang.txt"
            # File Access mode [Read Mode]
            accessMode = "r"
            with open(fileName, accessMode) as myCSVfile:
                # Reading file as CSV with delimiter as "=", so that 
                # abbreviation are stored in row[0] and phrases in row[1]
                dataFromFile = csv.reader(myCSVfile, delimiter="=")
                # Removing Special Characters.
                _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
                for row in dataFromFile:
                    # Check if selected word matches short forms[LHS] in text file.
                    if _str.upper() == row[0]:
                        # If match found replace it with its appropriate phrase in text file.
                        input_text[j] = row[1]
                myCSVfile.close()
            j = j + 1
        
        return(' '.join(input_text))
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords)
                    .apply(self.stemming))
        return clean_X
    
    def transform_no_stem(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords))
        return clean_X

# Read in Data and Create Train and Test Sets

In [4]:
tweets = pd.read_csv('https://github.com/anilkeshwani/StatLearnProj/raw/master/Iason/climate_change_tweets_sample-2020-05-16-17-57.csv')
tweets.head()

,username,user_handle,date,retweets,favorites,text,label
0,WWF Climate & Energy,climateWWF,2020-04-28,11,22,Economic recovery and national climate pledges...,0
1,WWF Climate & Energy,climateWWF,2020-04-22,6,16,"In this difficult time, it’s hard to connect w...",0
2,WWF Climate & Energy,climateWWF,2020-04-01,43,69,"The decision to postpone # COP26, is unavoidab...",0
3,WWF Climate & Energy,climateWWF,2020-03-30,24,30,Japan - the world’s fifth largest emitter of g...,0
4,WWF Climate & Energy,climateWWF,2020-03-30,22,40,How can countries include # NatureBasedSolutio...,0


In [5]:
ct = CleanText()
tweets["text"] = ct.fit_transform(tweets.text)
tweets = tweets.loc[(~tweets.text.isnull()), :]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(tweets.text, tweets.label, 
                                                    test_size=0.2, random_state=17, 
                                                    shuffle=True) # explicit default

[print(dat.head(3), dat.shape, end="\n"*2) for dat in [X_train, X_test, Y_train, Y_test]];

3641     insect integr part life environ increas heat c...
11837                                  believ climat chang
8450              climat model need worri programm set run
Name: text, dtype: object (14407,)

8375          climat scienc settl sustain stagnant
6110          call climat plan good start joebiden
16330    obama govern need save planet global warm
Name: text, dtype: object (3602,)

3641     0
11837    1
8450     1
Name: label, dtype: int64 (14407,)

8375     1
6110     0
16330    1
Name: label, dtype: int64 (3602,)



## **Logistic Regression**

- Logistic Regression is the appropriate regression analysis to conduct when the dependent variable is binary 
- From the results shown later we can see that the results given by different text representations doesn-t really differ. However the best results are given by the Bag of Words representation using Bigrams.

Hyper-parameters : 
- penalty : {l1, l2, elasticnet} (The norm used in penalization)
- solver : {'liblinear, 'saga', lbfgs} (Algorithm use in the optimization problem)
- l1_ratio : float (Only in the case of elasticnet)
- multi_class : SET TO 'ovr'
- max_iter : SET TO 1000
- C: float (Results showed that the best one is the default one (C=1))

# **Word Vectorisations**

- Bag of Words (BOW) Binary ("One-Hot") Representation

In [7]:
# Bag of Words Representation (One Hot, i.e. binary)

BOW_vectorizer = CountVectorizer(stop_words = 'english', 
                                 binary=True, # Creates 0/1 "One Hot" vector; 
                                              # np.unique(BOW_train.toarray())
                                 min_df = 10)
BOW_vectorizer.fit(X_train)
BOW_train = BOW_vectorizer.transform(X_train)
BOW_test = BOW_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_BOW = MaxAbsScaler()
BOW_train_scaled = scaler_BOW.fit_transform(BOW_train)
BOW_test_scaled = scaler_BOW.transform(BOW_test)

- Bag of Words with Frequencies Representation (FBOW)

In [8]:
# Bag of Words Representation (Frequencies; binary=False)

FBOW_vectorizer = CountVectorizer(stop_words = 'english', 
                                  binary=False, # Creates Word Frequency Vector; 
                                                # # np.unique(FBOW_train.toarray())
                                  min_df = 10)
FBOW_vectorizer.fit(X_train)
FBOW_train = FBOW_vectorizer.transform(X_train)
FBOW_test = FBOW_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_FBOW = MaxAbsScaler()
FBOW_train_scaled = scaler_FBOW.fit_transform(FBOW_train)
FBOW_test_scaled = scaler_FBOW.transform(FBOW_test)

- Bag of Words Bigrams (Bigrams)

In [9]:
bigram_vectorizer = CountVectorizer(stop_words = 'english', 
                                    binary=True, 
                                    min_df = 10,
                                    ngram_range = (1,2)) # create bigrams
bigram_vectorizer.fit(X_train)

bigram_train = bigram_vectorizer.transform(X_train)
bigram_test = bigram_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_bigram = MaxAbsScaler()
bigram_train_scaled = scaler_bigram.fit_transform(bigram_train)
bigram_test_scaled = scaler_bigram.transform(bigram_test)

- Term Frequency–Inverse Document Frequency Representation (tf-idf)

In [10]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', 
                                   min_df=10) # used for now for consistency
tfidf_vectorizer.fit(X_train)
tfidf_train = tfidf_vectorizer.transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# Construct Scaled Datasets

scaler_tfidf = MaxAbsScaler()
tfidf_train_scaled = scaler_tfidf.fit_transform(tfidf_train)
tfidf_test_scaled = scaler_tfidf.transform(tfidf_test)

# **Modelling** 
- Logistic Regression : BOW Representation

In [11]:
clf_BoW = LogisticRegression(C = 1, max_iter = 1000, multi_class = "ovr")
clf_BoW.fit(BOW_train_scaled, Y_train)
pred = clf_BoW.predict(BOW_test_scaled)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_BoW.predict_proba(BOW_test_scaled)[:, 1])

print('-- Logistic Regression : BoW Scaled -- ')
print()
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print()

clf_BoW.fit(BOW_train, Y_train)
pred = clf_BoW.predict(BOW_test)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_BoW.predict_proba(BOW_test)[:, 1])

print('-- Logistic Regression : BoW Non Scaled -- ')
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print('--------------')
print()

-- Logistic Regression : BoW Scaled -- 

Accuracy: 0.9098
Precision: 0.9175
Recall: 0.9317
f1_score: 0.9246
f2_score: 0.9288
f5_score: 0.9288
Auc score : 0.9687
--------------

-- Logistic Regression : BoW Non Scaled -- 
Accuracy: 0.9098
Precision: 0.9175
Recall: 0.9317
f1_score: 0.9246
f2_score: 0.9288
f5_score: 0.9288
Auc score : 0.9687
--------------



- Logistic Regression : Bigrams Representation

In [14]:
clf_BBoW = LogisticRegression(C = 1, max_iter = 1000, multi_class = "ovr", solver = "saga")
clf_BBoW.fit(bigram_train_scaled, Y_train)
pred = clf_BBoW.predict(bigram_test_scaled)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_BBoW.predict_proba(bigram_test_scaled)[:, 1])

print('-- Logistic Regression : Bigrams Scaled -- ')
print()
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print()

clf_BBoW.fit(bigram_train, Y_train)
pred = clf_BBoW.predict(bigram_test)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_BBoW.predict_proba(bigram_test)[:, 1])

print('-- Logistic Regression : Bigrams Non Scaled -- ')
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print('--------------')
print()

-- Logistic Regression : Bigrams Scaled -- 

Accuracy: 0.9203
Precision: 0.9267
Recall: 0.9401
f1_score: 0.9334
f2_score: 0.9374
f5_score: 0.9374
Auc score : 0.9723

-- Logistic Regression : Bigrams Non Scaled -- 
Accuracy: 0.9203
Precision: 0.9267
Recall: 0.9401
f1_score: 0.9334
f2_score: 0.9374
f5_score: 0.9374
Auc score : 0.9723
--------------



- Logistic Regression : FBOW Representation

In [15]:
clf_FBoW = LogisticRegression(C = 1, max_iter = 1000, multi_class = "ovr", solver = "liblinear")
clf_FBoW.fit(FBOW_train_scaled, Y_train)
pred = clf_FBoW.predict(FBOW_test_scaled)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_FBoW.predict_proba(FBOW_test_scaled)[:, 1])

print('-- Logistic Regression : FBoW Scaled -- ')
print()
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print()

clf_FBoW.fit(FBOW_train, Y_train)
pred = clf_FBoW.predict(FBOW_test)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_FBoW.predict_proba(FBOW_test)[:, 1])

print('-- Logistic Regression : FBoW Non Scaled -- ')
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print('--------------')
print()

-- Logistic Regression : FBoW Scaled -- 

Accuracy: 0.9117
Precision: 0.9201
Recall: 0.9322
f1_score: 0.9261
f2_score: 0.9297
f5_score: 0.9297
Auc score : 0.9702

-- Logistic Regression : FBoW Non Scaled -- 
Accuracy: 0.91
Precision: 0.9187
Recall: 0.9308
f1_score: 0.9247
f2_score: 0.9283
f5_score: 0.9283
Auc score : 0.969
--------------



- Logistic Regression : TF-IDF Representation

In [16]:
clf_TFIDF = LogisticRegression(C = 1, l1_ratio = 0.2,  penalty = "elasticnet", max_iter = 1000, multi_class = "ovr", solver = "saga")
clf_TFIDF.fit(tfidf_train_scaled, Y_train)
pred = clf_TFIDF.predict(tfidf_test_scaled)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_TFIDF.predict_proba(tfidf_test_scaled)[:, 1])

print('-- Logistic Regression : TF-IDF Scaled -- ')
print()
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print()

clf_TFIDF.fit(tfidf_train, Y_train)
pred = clf_TFIDF.predict(tfidf_test)
accuracy = accuracy_score(Y_test,pred)
precision = precision_score(Y_test,pred)
recall = recall_score(Y_test,pred)
f1 = f1_score(Y_test,pred)
f2 = fbeta_score(Y_test,pred,beta=2)
f5 = fbeta_score(Y_test,pred,beta=2)
auc_score = roc_auc_score(Y_test, clf_TFIDF.predict_proba(tfidf_test)[:, 1])

print('-- Logistic Regression : TF-IDF Non Scaled -- ')
print('Accuracy: {}'.format(round(accuracy,4)))
print('Precision: {}'.format(round(precision,4)))
print('Recall: {}'.format(round(recall,4)))
print('f1_score: {}'.format(round(f1,4)))
print('f2_score: {}'.format(round(f2,4)))
print('f5_score: {}'.format(round(f5,4)))
print("Auc score : {}".format(round(auc_score,4)))
print('--------------')
print()

-- Logistic Regression : TF-IDF Scaled -- 

Accuracy: 0.9153
Precision: 0.9257
Recall: 0.9322
f1_score: 0.9289
f2_score: 0.9309
f5_score: 0.9309
Auc score : 0.9702

-- Logistic Regression : TF-IDF Non Scaled -- 
Accuracy: 0.9134
Precision: 0.9243
Recall: 0.9303
f1_score: 0.9273
f2_score: 0.9291
f5_score: 0.9291
Auc score : 0.9686
--------------

